In [1]:
import pandas as pd
import datetime as dt
import numpy as np

pd.options.mode.chained_assignment = None  # default='warn'

# Consigna

Vamos a continuar trabajando con el dataset de la encuesta de sueldos en el sector tecnológico.

En esta oportunidad vamos a analizar la calidad de la información brindada. Nos interesa saber:

- si existen registros duplicados

- si existen datos faltantes: analizar especificamente las variables *tenes_algun_tipo_de_discapacidad* y *salario_mensual_neto_en_tu_moneda_local*; nos interesa imputar los faltantes en ambos casos

- si nuesta información es consistente tanto interna como externamente

Para el ultimo punto se sugiere:

- cruzar la edad con los años de experiencia (validez interna)

- cruzar la experiencia con los años en la experiencia actual (validez interna)

- comparar salario bruto con salario neto (validez interna)

- cruzar el sueldo percibido con el sueldo mínimo vigente (validez externa). TIP: usar solo los casos de Argentina (salario mínimo vigente para 2020: $16.875,0)

In [2]:
sueldos = pd.read_csv("https://raw.githubusercontent.com/eea-uba/EEA-2020/master/Fuentes/encuesta_sueldos_sysarmy_1s2020.csv")
sueldos.head()

,timestamp,me_identifico,tengo,estoy_trabajando_en,donde_estas_trabajando,anos_de_experiencia,anos_en_la_empresa_actual,anos_en_el_puesto_actual,gente_a_cargo,nivel_de_estudios_alcanzado,...,sos_miembro_de_alguna_comunidad_it,cantidad_de_empleados,actividad_principal,la_recomendas_como_un_buen_lugar_para_trabajar,como_calificas_las_politicas_de_diversidad_e_inclusion,a_cuantos_kilometros_de_tu_casa_queda_la_oficina,beneficios_extra,habias_respondido_nuestra_encuesta_en_ediciones_anteriores,cuales_consideras_que_son_las_mejores_empresas_de_it_para_trabajar_en_este_momentoen_tu_ciudad,sueldo_dolarizado
0,1/31/2020 6:50:26,Hombre,32,Argentina,Ciudad Autónoma de Buenos Aires,10,3,1,0,Secundario,...,NaN,10001+,Producto basado en Software,8,10,6,"Capacitaciones y/o cursos, Comidas pagas / sub...",No,cognizant,False
1,1/28/2020 9:27:48,Hombre,30,Argentina,Ciudad Autónoma de Buenos Aires,3,3,3,0,Terciario,...,NaN,51-100,Otras industrias,7,3,35,Capacitaciones y/o cursos,No,mercadolibre,False
2,1/29/2020 16:54:29,Mujer,40,Argentina,Ciudad Autónoma de Buenos Aires,15,3,3,1,Universitario,...,NaN,11-50,Servicios / Consultoría de Software / Digital,8,8,30,"Abono de celular y/o Internet, Capacitaciones ...",No,NaN,False
3,2/1/2020 5:46:25,Mujer,36,Argentina,Ciudad Autónoma de Buenos Aires,10,3,3,0,Universitario,...,NaN,51-100,Otras industrias,7,5,4,Viáticos,No,NaN,False
4,2/1/2020 17:51:21,Hombre,29,Argentina,Ciudad Autónoma de Buenos Aires,10,2,2,0,Universitario,...,NaN,11-50,Otras industrias,5,8,7,Ninguna de las anteriores,No,NaN,False


## 1) Duplicados

In [3]:
# Validamos si hay registros duplicados
sueldos.duplicated().sum()

0

## 2) Faltantes

In [4]:
sueldos.isnull().sum()

timestamp                                                                                            0
me_identifico                                                                                        0
tengo                                                                                                0
estoy_trabajando_en                                                                                  0
donde_estas_trabajando                                                                               0
anos_de_experiencia                                                                                  0
anos_en_la_empresa_actual                                                                            0
anos_en_el_puesto_actual                                                                             0
gente_a_cargo                                                                                        0
nivel_de_estudios_alcanzado                                              

In [5]:
# Analizamos los posibles valores de la variable tenes_algun_tipo_de_discapacidad
sueldos.tenes_algun_tipo_de_discapacidad.unique()

array([nan, 'Visual', 'Auditiva', 'Motriz', 'Verbal', 'Epilepsia',
       'Diabetes ', '.', 'No ninguna', 'Mental', 'Dislexia',
       'uso anteojos', 'N8bguna', 'Edad', 'n9', 'Programador',
       'Faltó la opción Ninguna', 'que te importa gil', 'Depresión',
       'Visceral', 'Epilepsia ', 'ADHD (ADD) y Tourette', 'Tartamudez',
       'Problemas en la voz (Disfonia)', 'N0', 'Migraña crónica ',
       'Debería haber opciones de respetar su privacidad',
       'Atigmatismo y miopía, pero puedo ver :)', 'Posible Asperger',
       'Obesidad', 'tendinitis en las manos', 'Migrañas Crónicas'],
      dtype=object)

In [6]:
# imputamos como "no" a aquellos valores sin responder
sueldos['tenes_algun_tipo_de_discapacidad_fill'] = sueldos['tenes_algun_tipo_de_discapacidad'].fillna('NO')

In [7]:
sueldos.tenes_algun_tipo_de_discapacidad_fill.isnull().sum()

0

In [8]:
# Para la variable salario_mensual_neto_en_tu_moneda_local, imputamos la mediana en los datos faltantes
mediana_sueldo_neto = sueldos['salario_mensual_neto_en_tu_moneda_local'].median()

sueldos['salario_mensual_neto_en_tu_moneda_local_fill'] = sueldos['salario_mensual_neto_en_tu_moneda_local'].fillna(mediana_sueldo_neto)

In [9]:
mediana_sueldo_neto

60000.0

In [10]:
sueldos.salario_mensual_neto_en_tu_moneda_local_fill.isnull().sum()

0

## 3) *Sanity Check*

* cruzar la edad con los años de experiencia (validez interna)

* cruzar la experiencia con los años en la experiencia actual (validez interna)

* comparar salario bruto con salario neto (validez interna)

* cruzar el sueldo percibido con el sueldo mínimo vigente (validez externa). TIP: usar solo los casos de Argentina (salario mínimo vigente para 2020: $16.875,0)

Para esta parte vamos a utilizar unicamente los salarios de Argentina.

In [11]:
sueldos = sueldos[sueldos.estoy_trabajando_en == 'Argentina']

### a) cruzar la edad con los años de experiencia (validez interna)

In [12]:
sueldos.tengo.head()

0    32
1    30
2    40
3    36
4    29
Name: tengo, dtype: int64

In [13]:
# calculo la edad en que la persona empezo a trabajar de acuerdo a lo declarado
sueldos['empieza_trabajar'] = sueldos.tengo - sueldos.anos_de_experiencia

In [14]:
print(min(sueldos['empieza_trabajar']), max(sueldos['empieza_trabajar']))

-37 57


In [15]:
# marcamos aquellos casos en los cuales la edad en la cual empezó a trabajar es inferior a los 17 años
sueldos['inconsistencia_experiencia'] = np.where(sueldos['empieza_trabajar'] < 17, 1, 0)

In [16]:
sueldos[sueldos['inconsistencia_experiencia'] == 1].head()

,timestamp,me_identifico,tengo,estoy_trabajando_en,donde_estas_trabajando,anos_de_experiencia,anos_en_la_empresa_actual,anos_en_el_puesto_actual,gente_a_cargo,nivel_de_estudios_alcanzado,...,como_calificas_las_politicas_de_diversidad_e_inclusion,a_cuantos_kilometros_de_tu_casa_queda_la_oficina,beneficios_extra,habias_respondido_nuestra_encuesta_en_ediciones_anteriores,cuales_consideras_que_son_las_mejores_empresas_de_it_para_trabajar_en_este_momentoen_tu_ciudad,sueldo_dolarizado,tenes_algun_tipo_de_discapacidad_fill,salario_mensual_neto_en_tu_moneda_local_fill,empieza_trabajar,inconsistencia_experiencia
23,1/30/2020 12:10:43,Hombre,24,Argentina,Ciudad Autónoma de Buenos Aires,8,6,2,0,Terciario,...,3,7,"Capacitaciones y/o cursos, Cuota para universi...",No,-,False,NO,31300.0,16,1
206,2/1/2020 0:18:11,Hombre,45,Argentina,Ciudad Autónoma de Buenos Aires,30,3,3,10,Universitario,...,8,20,"Capacitaciones y/o cursos, Vacaciones flexible...",No,Mulesoft,False,NO,75000.0,15,1
207,2/1/2020 9:55:12,Hombre,40,Argentina,Ciudad Autónoma de Buenos Aires,24,1,1,0,Secundario,...,10,35,Horarios flexibles,No,NaN,False,NO,74000.0,16,1
222,2/2/2020 20:24:25,Hombre,39,Argentina,Buenos Aires,25,5,5,0,Universitario,...,10,5,"Abono de celular y/o Internet, Comidas pagas /...",No,Watcha S.A.S.,False,NO,5000.0,14,1
291,1/1/2020 12:14:23,Mujer,27,Argentina,Córdoba,25,25,1,0,Universitario,...,7,2,"Capacitaciones y/o cursos, Clases de idiomas, ...",No,NaN,True,NO,45000.0,2,1


### b) cruzar la experiencia con los años en la experiencia actual (validez interna)

In [17]:
sueldos['inconsistencia_experiencia_trabajo_actual'] = np.where(sueldos['anos_de_experiencia'] < sueldos['anos_en_la_empresa_actual'], 1, 0)

In [18]:
sueldos[sueldos['inconsistencia_experiencia_trabajo_actual'] == 1].head()

,timestamp,me_identifico,tengo,estoy_trabajando_en,donde_estas_trabajando,anos_de_experiencia,anos_en_la_empresa_actual,anos_en_el_puesto_actual,gente_a_cargo,nivel_de_estudios_alcanzado,...,a_cuantos_kilometros_de_tu_casa_queda_la_oficina,beneficios_extra,habias_respondido_nuestra_encuesta_en_ediciones_anteriores,cuales_consideras_que_son_las_mejores_empresas_de_it_para_trabajar_en_este_momentoen_tu_ciudad,sueldo_dolarizado,tenes_algun_tipo_de_discapacidad_fill,salario_mensual_neto_en_tu_moneda_local_fill,empieza_trabajar,inconsistencia_experiencia,inconsistencia_experiencia_trabajo_actual
21,12/31/2019 1:05:01,Hombre,32,Argentina,Buenos Aires,7,13,7,3,Universitario,...,9,"Abono de celular y/o Internet, Vacaciones flex...",Sí,Mercadolibre,False,NO,58000.0,25,0,1
25,2/1/2020 14:35:24,Mujer,22,Argentina,Ciudad Autónoma de Buenos Aires,2,3,2,0,Universitario,...,15,"Capacitaciones y/o cursos, Guardería, Lactario...",No,0,False,NO,37000.0,20,0,1
26,2/1/2020 14:40:00,Mujer,52,Argentina,Ciudad Autónoma de Buenos Aires,22,23,10,0,Secundario,...,7,Licencia de paternidad/maternidad extendida,No,NaN,False,NO,24000.0,30,0,1
42,1/9/2020 12:20:46,Hombre,26,Argentina,Ciudad Autónoma de Buenos Aires,6,8,4,0,Universitario,...,15,Capacitaciones y/o cursos,No,"EY, JP Morgan, Accenture, Techint",False,NO,70000.0,20,0,1
63,2/1/2020 14:43:05,Mujer,43,Argentina,Ciudad Autónoma de Buenos Aires,3,13,3,1,Terciario,...,8,"Comidas pagas / subvencionadas, Horarios flexi...",No,"Despegar, telefonica",False,NO,58.0,40,0,1


### c) comparar salario bruto con salario neto (validez interna)

In [19]:
sueldos['inconsistencia_salario_bruto_neto'] = np.where(sueldos['salario_mensual_bruto_en_tu_moneda_local'] < sueldos['salario_mensual_neto_en_tu_moneda_local'], 1, 0)

In [20]:
sueldos[sueldos['inconsistencia_salario_bruto_neto'] == 1].head()

,timestamp,me_identifico,tengo,estoy_trabajando_en,donde_estas_trabajando,anos_de_experiencia,anos_en_la_empresa_actual,anos_en_el_puesto_actual,gente_a_cargo,nivel_de_estudios_alcanzado,...,beneficios_extra,habias_respondido_nuestra_encuesta_en_ediciones_anteriores,cuales_consideras_que_son_las_mejores_empresas_de_it_para_trabajar_en_este_momentoen_tu_ciudad,sueldo_dolarizado,tenes_algun_tipo_de_discapacidad_fill,salario_mensual_neto_en_tu_moneda_local_fill,empieza_trabajar,inconsistencia_experiencia,inconsistencia_experiencia_trabajo_actual,inconsistencia_salario_bruto_neto
13,2/2/2020 19:01:32,Mujer,42,Argentina,Ciudad Autónoma de Buenos Aires,15,11,11,0,Universitario,...,Guardería,No,NaN,False,NO,24000.0,27,0,0,1
24,1/31/2020 18:45:21,Mujer,22,Argentina,Ciudad Autónoma de Buenos Aires,1,1,1,0,Universitario,...,Horarios flexibles,No,NaN,False,NO,18000.0,21,0,0,1
76,1/31/2020 20:41:51,Hombre,24,Argentina,Ciudad Autónoma de Buenos Aires,1,0,0,0,Terciario,...,"Capacitaciones y/o cursos, Descuentos varios (...",No,no se,False,NO,392175.0,23,0,0,1
97,12/15/2019 0:56:50,Hombre,43,Argentina,Santa Fe,14,11,2,0,Universitario,...,"Capacitaciones y/o cursos, Horarios flexibles,...",Sí,Nemogroup,False,NO,75000.0,29,0,0,1
126,1/4/2020 15:59:23,Hombre,27,Argentina,Ciudad Autónoma de Buenos Aires,7,1,1,5,Universitario,...,"Capacitaciones y/o cursos, Gimnasios gratuitos...",No,NaN,False,NO,74000.0,20,0,0,1


### d) cruzar el sueldo percibido con el sueldo mínimo vigente (validez externa)

In [21]:
sueldos['inconsistencia_salario_minimo'] = np.where(sueldos['salario_mensual_bruto_en_tu_moneda_local'] < 16875, 1, 0)

In [22]:
sueldos[sueldos['inconsistencia_salario_minimo'] == 1].head()

,timestamp,me_identifico,tengo,estoy_trabajando_en,donde_estas_trabajando,anos_de_experiencia,anos_en_la_empresa_actual,anos_en_el_puesto_actual,gente_a_cargo,nivel_de_estudios_alcanzado,...,habias_respondido_nuestra_encuesta_en_ediciones_anteriores,cuales_consideras_que_son_las_mejores_empresas_de_it_para_trabajar_en_este_momentoen_tu_ciudad,sueldo_dolarizado,tenes_algun_tipo_de_discapacidad_fill,salario_mensual_neto_en_tu_moneda_local_fill,empieza_trabajar,inconsistencia_experiencia,inconsistencia_experiencia_trabajo_actual,inconsistencia_salario_bruto_neto,inconsistencia_salario_minimo
0,1/31/2020 6:50:26,Hombre,32,Argentina,Ciudad Autónoma de Buenos Aires,10,3,1,0,Secundario,...,No,cognizant,False,NO,40.0,22,0,0,0,1
7,2/2/2020 11:46:48,Hombre,24,Argentina,Ciudad Autónoma de Buenos Aires,2,2,2,0,Secundario,...,No,NaN,False,NO,22.0,22,0,0,0,1
13,2/2/2020 19:01:32,Mujer,42,Argentina,Ciudad Autónoma de Buenos Aires,15,11,11,0,Universitario,...,No,NaN,False,NO,24000.0,27,0,0,1,1
20,1/29/2020 13:57:21,Hombre,22,Argentina,Ciudad Autónoma de Buenos Aires,1,1,1,0,Universitario,...,No,NaN,False,NO,10000.0,21,0,0,0,1
24,1/31/2020 18:45:21,Mujer,22,Argentina,Ciudad Autónoma de Buenos Aires,1,1,1,0,Universitario,...,No,NaN,False,NO,18000.0,21,0,0,1,1


### Validación final

In [23]:
registros_raw = len(sueldos)
registros_clean = len(sueldos[(sueldos.inconsistencia_experiencia < 1) &
                  (sueldos.inconsistencia_experiencia_trabajo_actual < 1) &
                  (sueldos.inconsistencia_salario_bruto_neto < 1) &
                  (sueldos.inconsistencia_salario_minimo < 1)])
diferencia = registros_raw - registros_clean

In [24]:
print("""Previo a la limipeza, el dataset presenta {} registros para Argentina. Luego de aplicar los filtros para 
quitar los registros con inconsistencias, se conservan {} registros habiendo eliminado {} observaciones.""".format(registros_raw, registros_clean, diferencia))

Previo a la limipeza, el dataset presenta 5982 registros para Argentina. Luego de aplicar los filtros para 
quitar los registros con inconsistencias, se conservan 5358 registros habiendo eliminado 624 observaciones.
